## 🎨 NeMo Data Designer 101: The Basics

#### 📚 What you'll learn

This notebook demonstrates the basics of Data Designer by generating a simple product review dataset.


### 📦 Import the essentials

- The `data_designer` module of `nemo_microservices` exposes Data Designer's high-level SDK.

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    CategorySamplerParams,
    DataDesigner,
    DataDesignerConfigBuilder,
    InferenceParameters,
    LLMTextColumnConfig,
    ModelConfig,
    PersonSamplerParams,
    SamplerColumnConfig,
    SamplerType,
    SubcategorySamplerParams,
    UniformSamplerParams,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is responsible for submitting generation requests to the microservice.


In [2]:
NEMO_MICROSERVICES_BASE_URL = "http://localhost:8080"

data_designer = DataDesigner()

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see [the model config docs](https://docs.nvidia.com/nemo/microservices/latest/design-synthetic-data-from-scratch-or-seeds/configure-models.html) for more details).

- By default, the microservice uses [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the microservice deployment configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nvidia-nemotron-nano-9b-v2"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v2"

# This sets reasoning to False for the nemotron-nano-v2 model.
SYSTEM_PROMPT = "/no_think"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.5,
            top_p=1.0,
            max_tokens=1024,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🎲 Getting started with sampler columns

- Sampler columns offer non-LLM based generation of synthetic data.

- They are particularly useful for **steering the diversity** of the generated data, as we demonstrate below.

<br>

You can view available samplers using the config builder's `info` property:


In [7]:
config_builder.info.display("SAMPLERS")

─────────────────────────────────────────── NeMo Data Designer Samplers ───────────────────────────────────────────

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Type              ┃ Parameter                     ┃ Data Type                     ┃ Required ┃ Constraints      ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ bernoulli         │ p                             │ number                        │    ✓     │ >= 0.0, <= 1.0   │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ bernoulli_mixture │ p                             │ number                        │    ✓     │ >= 0.0, <= 1.0   │
│                   │ dist_name                     │ string                        │    ✓     │                  │
│                   │ dist_params                   │ dict                          │    ✓     │                  │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ binomial          │ n                             │ integer                       │    ✓     │                  │
│                   │ p                             │ number                        │    ✓     │ >= 0.0, <= 1.0   │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ category          │ values                        │ string[] | integer[] |        │    ✓     │ len > 1          │
│                   │                               │ number[]                      │          │                  │
│                   │ weights                       │ number[] | null               │          │                  │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ datetime          │ start                         │ string                        │    ✓     │                  │
│                   │ end                           │ string                        │    ✓     │                  │
│                   │ unit                          │ string                        │          │                  │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ gaussian          │ mean                          │ number                        │    ✓     │                  │
│                   │ stddev                        │ number                        │    ✓     │                  │
│                   │ decimal_places                │ integer | null                │          │                  │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ person            │ locale                        │ string                        │          │                  │
│                   │ sex                           │ string | null                 │          │                  │
│                   │ city                          │ string | string[] | null      │          │                  │
│                   │ age_range                     │ integer[]                     │          │ len > 2, len < 2 │
│                   │ state                         │ string | string[] | null      │          │                  │
│                   │ with_synthetic_personas       │ boolean                       │          │                  │
│                   │ sample_dataset_when_available │ boolean                       │          │                  │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ poisson           │ mean                          │ number                        │    ✓     │                  │
├───────────────────┼───────────────────────────────┼──

Let's start designing our product review dataset by adding product category and subcategory columns.


In [8]:
config_builder.add_column(
    SamplerColumnConfig(
        name="product_category",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "Electronics",
                "Clothing",
                "Home & Kitchen",
                "Books",
                "Home Office",
            ],
        ),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="product_subcategory",
        sampler_type=SamplerType.SUBCATEGORY,
        params=SubcategorySamplerParams(
            category="product_category",
            values={
                "Electronics": [
                    "Smartphones",
                    "Laptops",
                    "Headphones",
                    "Cameras",
                    "Accessories",
                ],
                "Clothing": [
                    "Men's Clothing",
                    "Women's Clothing",
                    "Winter Coats",
                    "Activewear",
                    "Accessories",
                ],
                "Home & Kitchen": [
                    "Appliances",
                    "Cookware",
                    "Furniture",
                    "Decor",
                    "Organization",
                ],
                "Books": [
                    "Fiction",
                    "Non-Fiction",
                    "Self-Help",
                    "Textbooks",
                    "Classics",
                ],
                "Home Office": [
                    "Desks",
                    "Chairs",
                    "Storage",
                    "Office Supplies",
                    "Lighting",
                ],
            },
        ),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="target_age_range",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(values=["18-25", "25-35", "35-50", "50-65", "65+"]),
    )
)

# Optionally validate that the columns are configured correctly.
config_builder.validate()

[17:21:36] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    sampler_columns: ['product_category', 'product_subcategory', 'target_age_range']
)

Next, let's add samplers to generate data related to the customer and their review.


In [12]:
config_builder.add_column(
    SamplerColumnConfig(
        name="customer",
        sampler_type=SamplerType.PERSON,
        params=PersonSamplerParams(age_range=[18, 70], locale="en_GB"),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="number_of_stars",
        sampler_type=SamplerType.UNIFORM,
        params=UniformSamplerParams(low=1, high=5),
        convert_to="int",  # Convert the sampled float to an integer.
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="review_style",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["rambling", "brief", "detailed", "structured with bullet points"],
            weights=[1, 2, 2, 1],
        ),
    )
)

config_builder.validate()

[17:22:38] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    sampler_columns: [
        "product_category",
        "product_subcategory",
        "target_age_range",
        "customer",
        "number_of_stars",
        "review_style"
    ]
    llm_text_columns: ['product_name', 'customer_review']
)

## 🦜 LLM-generated columns

- The real power of Data Designer comes from leveraging LLMs to generate text, code, and structured data.

- When prompting the LLM, we can use Jinja templating to reference other columns in the dataset.

- As we see below, nested json fields can be accessed using dot notation.


In [13]:
config_builder.add_column(
    LLMTextColumnConfig(
        name="product_name",
        prompt=(
            "You are a helpful assistant that generates product names. DO NOT add quotes around the product name.\n\n"
            "Come up with a creative product name for a product in the '{{ product_category }}' category, focusing "
            "on products related to '{{ product_subcategory }}'. The target age range of the ideal customer is "
            "{{ target_age_range }} years old. Respond with only the product name, no other text."
        ),
        system_prompt=SYSTEM_PROMPT,
        model_alias=MODEL_ALIAS,
    )
)

config_builder.add_column(
    LLMTextColumnConfig(
        name="customer_review",
        prompt=(
            "You are a customer named {{ customer.first_name }} from {{ customer.city }}, {{ customer.state }}. "
            "You are {{ customer.age }} years old and recently purchased a product called {{ product_name }}. "
            "Write a review of this product, which you gave a rating of {{ number_of_stars }} stars. "
            "The style of the review should be '{{ review_style }}'."
        ),
        system_prompt=SYSTEM_PROMPT,
        model_alias=MODEL_ALIAS,
    )
)

config_builder.validate()

[17:22:38] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    sampler_columns: [
        "product_category",
        "product_subcategory",
        "target_age_range",
        "customer",
        "number_of_stars",
        "review_style"
    ]
    llm_text_columns: ['product_name', 'customer_review']
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [14]:
preview = data_designer.preview(config_builder)

[17:22:39] [INFO] 🔁 Preview generation in progress
[17:22:39] [INFO] ✅ Validation passed
[17:22:39] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[17:22:39] [INFO] 🩺 Running health checks for models...
[17:22:39] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[17:22:41] [INFO]   |-- ✅ Passed!
[17:22:41] [INFO] 🎲 Preparing samplers to generate 10 records across 6 columns
[17:22:41] [INFO] 📝 Preparing llm-text column generation
[17:22:41] [INFO]   |-- column name: 'product_name'
[17:22:41] [INFO]   |-- model config:
{
    "alias": "nemotron-nano-v2",
    "model": "nvidia/nvidia-nemotron-nano-9b-v2",
    "inference_parameters": {
        "temperature": 0.5,
        "top_p": 1.0,
        "max_tokens": 1024,
        "max_parallel_requests": 4,
        "timeout": null,
        "extra_body": null
    },
    "provider": "nvidia"
}
[17:22:41] [INFO] 🐙 Processing llm-text column 'product_name' with 4 c

In [15]:
# Run this cell multiple times to cycle through the 10 preview records.
preview.display_sample_record()

                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                ┃ Value                                                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_category    │ Home Office                                                                               │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ product_subcategory │ Office Supplies                                                                           │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ target_age_range    │ 65+                                                                                       │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ customer            │ {                                                                                         │
│                     │     'uuid': '60bc35b3-413f-497b-a5f4-64b71e6ac815',                                       │
│                     │     'locale': 'en_GB',                                                                    │
│                     │     'first_name': 'Jamie',                                                                │
│                     │     'last_name': 'Long',                                                                  │
│                     │     'middle_name': None,                                                                  │
│                     │     'sex': 'Male',                                                                        │
│                     │     'street_number': '84',                                                                │
│                     │     'street_name': 'Dixon road',                                                          │
│                     │     'city': 'New Jamesbury',                                                              │
│                     │     'state': None,                                                                        │
│                     │     'postcode': 'G9 8BE',                                                                 │
│                     │     'age': 30,                                                                            │
│                     │     'birth_date': '1995-07-26',                                                           │
│                     │     'country': 'Syrian Arab Republic',                                                    │
│                     │     'marital_status': 'widowed',                                                          │
│                     │     'education_level': 'secondary_education',                                             │
│                     │     'unit': '',                                                                           │
│                     │     'occupation': 'Accountant, chartered public finance',                                 │
│                     │     'phone_number': '(0909) 879 0518',                                                    │
│                     │     'bachelors_field': 'no_degree'                                                        │
│                     │ }                                                                                         │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ number_of_stars     │ 4                                                                                         │
├─────────────────────┼─────────────────────────────────

In [16]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,product_category,product_subcategory,target_age_range,customer,number_of_stars,review_style,product_name,customer_review
0,Home Office,Office Supplies,65+,{'uuid': '60bc35b3-413f-497b-a5f4-64b71e6ac815...,4,detailed,SeniorDeskEssentials\n,**Product Review: SeniorDeskEssentials – A Tho...
1,Books,Classics,18-25,{'uuid': '388aad06-c9f2-48a4-aa3c-3af206550fa2...,5,detailed,Timeless Tales\n,**Product Review: Timeless Tales – A 5-Star Ex...
2,Home Office,Lighting,18-25,{'uuid': '0679b85a-203d-42d4-9a55-35d5559cdabd...,3,detailed,LumenLounge\n,**Review Title:** A Mixed Experience with Lume...
3,Home Office,Desks,50-65,{'uuid': 'cc356033-209e-4915-85f8-544e0a95df3c...,5,brief,TimelessDesk\n,"**Review:** \nA sturdy, elegant desk that ble..."
4,Home Office,Office Supplies,25-35,{'uuid': '2e18b9a6-c765-4e7b-a7cd-85467ce56287...,5,detailed,SmartDeskEssentials\n,**Product Review: SmartDeskEssentials – A Game...
5,Home & Kitchen,Organization,65+,{'uuid': 'a142c515-be9f-40e8-b024-5fb9a30de7ce...,3,detailed,SeniorSpace Organizer\n,**Product Review: SeniorSpace Organizer** \n*...
6,Electronics,Headphones,35-50,{'uuid': '1c52a36e-fb4f-49d4-8683-1a5538a21e53...,1,detailed,SoundSage\n,**Review Title: A Disappointing Waste of Time ...
7,Home & Kitchen,Organization,50-65,{'uuid': '58a33403-0f81-4356-a33d-a2a13f9acd52...,2,detailed,NestEase Organizer\n,**Product Review: NestEase Organizer (2 Stars)...
8,Home Office,Chairs,50-65,{'uuid': '6ca5c7ac-cf15-4111-b58e-3d261ef2b3bb...,1,brief,ErgoComfort\n,**Review:** \nErgoComfort is uncomfortable an...
9,Home Office,Desks,50-65,{'uuid': '3f3f251a-557c-4edd-af43-77f83d161985...,5,rambling,HeritageDesk\n,"Oh my goodness, let me just start by saying th..."


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [17]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 8                               │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                      ┃        data type ┃               number unique values ┃         sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ product_category                 │           string │                          4 (40.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ product_subcategory              │           string │                          7 (70.0%) │          subcategory │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ target_age_range                 │           string │                          5 (50.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ customer                         │             dict │                        10 (100.0%) │               person │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ number_of_stars                  │              int │                          5 (50.0%) │              uniform │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ review_style                     │           string │                          3 (30.0%) │             category │
└──────────────────────────────────┴──────────────────┴────────────────────────────────────┴──────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_name          │        string │                10 (100.0%) │      77.0 +/- 0.5 │            4.5 +/- 0.5 │
├───────────────────────┼───────────────┼─────────────────

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [18]:
results = data_designer.create(config_builder, num_records=20)

[17:23:40] [INFO] 🎨 Creating Data Designer dataset
[17:23:40] [INFO] ✅ Validation passed
[17:23:40] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[17:23:40] [INFO] 🩺 Running health checks for models...
[17:23:40] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[17:23:42] [INFO]   |-- ✅ Passed!
[17:23:42] [INFO] ⏳ Processing batch 1 of 1
[17:23:42] [INFO] 🎲 Preparing samplers to generate 20 records across 6 columns
[17:23:42] [INFO] 📝 Preparing llm-text column generation
[17:23:42] [INFO]   |-- column name: 'product_name'
[17:23:42] [INFO]   |-- model config:
{
    "alias": "nemotron-nano-v2",
    "model": "nvidia/nvidia-nemotron-nano-9b-v2",
    "inference_parameters": {
        "temperature": 0.5,
        "top_p": 1.0,
        "max_tokens": 1024,
        "max_parallel_requests": 4,
        "timeout": null,
        "extra_body": null
    },
    "provider": "nvidia"
}
[17:23:42] [INFO] 🐙 Proces

In [19]:
# Load the generated dataset as a pandas DataFrame.
dataset = results.load_dataset()

dataset.head()

,product_category,product_subcategory,target_age_range,customer,number_of_stars,review_style,product_name,customer_review
0,Clothing,Activewear,35-50,"{'age': 57, 'bachelors_field': 'no_degree', 'b...",1,detailed,UrbanMotionFlex,**Product Review: UrbanMotionFlex – A Disappoi...
1,Clothing,Activewear,50-65,"{'age': 68, 'bachelors_field': 'education', 'b...",2,brief,FreedomActive,Rating: 2 stars. FreedomActive didn’t meet e...
2,Home & Kitchen,Furniture,65+,"{'age': 20, 'bachelors_field': 'stem_related',...",1,brief,Timeless Nest,"**Review:** Disappointing. Poor quality, not w..."
3,Books,Self-Help,65+,"{'age': 39, 'bachelors_field': 'no_degree', 'b...",1,detailed,WiseWhispers,**Product Review: WiseWhispers (1 Star)** *B...
4,Electronics,Headphones,50-65,"{'age': 19, 'bachelors_field': 'business', 'bi...",1,brief,SoundSage,"**SoundSage: 1 star** Overpriced, poorly bui..."


In [20]:
# Load the analysis results into memory.
analysis = results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 20                              │ 8                               │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                      ┃        data type ┃               number unique values ┃         sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ product_category                 │           string │                          5 (25.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ product_subcategory              │           string │                         11 (55.0%) │          subcategory │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ target_age_range                 │           string │                          5 (25.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ customer                         │             dict │                        20 (100.0%) │               person │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ number_of_stars                  │              int │                          5 (25.0%) │              uniform │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ review_style                     │           string │                          4 (20.0%) │             category │
└──────────────────────────────────┴──────────────────┴────────────────────────────────────┴──────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_name          │        string │                 19 (95.0%) │      77.0 +/- 0.7 │            4.5 +/- 1.2 │
├───────────────────────┼───────────────┼─────────────────

## ⏭️ Next Steps

Now that you've seen the basics of Data Designer, check out the following notebooks to learn more about:

- [Structured outputs and jinja expressions](./2-structured-outputs-and-jinja-expressions.ipynb)

- [Seeding synthetic data generation with an external dataset](./3-seeding-with-a-dataset.ipynb)
